In [1]:
# first 5 cells are fix for running tensorflow with mixed precision for XLM-R model
import json
json_path = json.__file__
dirs = []
for dir_name in json_path.split('/'):
    dirs.append(dir_name)
    if dir_name.startswith('python'):
        break
python_lib_path = '/'.join(dirs)
python_lib_path

'/opt/conda/lib/python3.7'

In [2]:
# The fix is done for tensorflow 2.1. Very likely not working for other versions.
# However, the current environment should contain tf 2.1, so we don't reinstall tensorflow here.

# !pip uninstall -y tensorflow
# !pip install --upgrade tensorflow==2.1

lines = []
with open(python_lib_path + '/site-packages/tensorflow_core/python/keras/layers/normalization.py', "r", encoding="UTF-8") as fp:
    for line in fp:

        if line.strip() == "outputs = outputs * scale":
            
            line_modified = "        outputs = outputs * math_ops.cast(scale, outputs.dtype)" + "\n"
            lines.append(line_modified)

        elif line.strip() == "outputs = outputs + offset":
         
            line_modified = "        outputs = outputs + math_ops.cast(offset, outputs.dtype)" + "\n"
            lines.append(line_modified)

        elif line == "      scale, offset = _broadcast(self.gamma), _broadcast(self.beta)" + "\n":
           
            lines.append(line)
            lines.append("      scale = math_ops.cast(scale, inputs.dtype)" + "\n")
            lines.append("      offset = math_ops.cast(offset, inputs.dtype)" + "\n")            

        else:

            lines.append(line)

with open(python_lib_path + '/site-packages/tensorflow_core/python/keras/layers/normalization.py', "w", encoding="UTF-8") as fp:
    for line in lines:
        fp.write(line)

In [3]:
# The fix is done for tensorflow 2.1. Very likely not working for other versions.
# However, the current environment should contain tf 2.1, so we don't reinstall tensorflow here.

# !pip uninstall -y tensorflow
# !pip install --upgrade tensorflow==2.1

lines = []
with open(python_lib_path + '/site-packages/tensorflow_core/python/keras/layers/pooling.py', "r", encoding="UTF-8") as fp:
    for line in fp:

        if line.strip() == "inputs *= mask":
            
            lines.append("      mask = math_ops.cast(mask, inputs.dtype)" + "\n")
            lines.append(line)

        else:

            lines.append(line)

with open(python_lib_path + '/site-packages/tensorflow_core/python/keras/layers/pooling.py', "w", encoding="UTF-8") as fp:
    for line in lines:
        fp.write(line)

In [4]:
# !pip uninstall -y transformers
# !pip install --upgrade transformers==2.9.1

lines = []
with open(python_lib_path + '/site-packages/transformers/modeling_tf_bert.py', "r", encoding="UTF-8") as fp:
    for line in fp:

        if line.strip() == "embeddings = inputs_embeds + position_embeddings + token_type_embeddings":

            line_modified = "        embeddings = inputs_embeds + tf.cast(position_embeddings, dtype=inputs_embeds.dtype) + tf.cast(token_type_embeddings, dtype=inputs_embeds.dtype)" + "\n"
            lines.append(line_modified)

        elif line.strip() == "attention_scores = attention_scores / tf.math.sqrt(dk)":

            line_modified = "        attention_scores = attention_scores / tf.cast(tf.math.sqrt(dk), dtype=attention_scores.dtype)" + "\n"
            lines.append(line_modified)

        else:

            lines.append(line)

with open(python_lib_path + '/site-packages/transformers/modeling_tf_bert.py', "w", encoding="UTF-8") as fp:
    for line in lines:
        fp.write(line)


lines = []
with open(python_lib_path + '/site-packages/transformers/modeling_tf_bert.py', "r", encoding="UTF-8") as fp:
    for line in fp:

        if line.strip() == "cdf = 0.5 * (1.0 + tf.math.erf(x / tf.math.sqrt(2.0)))":

            line_modified = "    cdf = tf.cast(0.5, dtype=x.dtype) * (tf.cast(1.0, dtype=x.dtype) + tf.math.erf(x / tf.cast(tf.math.sqrt(2.0), dtype=x.dtype)))" + "\n"
            lines.append(line_modified)

        else:

            lines.append(line)

with open(python_lib_path + '/site-packages/transformers/modeling_tf_bert.py', "w", encoding="UTF-8") as fp:
    for line in lines:
        fp.write(line)

In [5]:
lines = []
with open(python_lib_path + '/site-packages/transformers/modeling_tf_distilbert.py', "r", encoding="UTF-8") as fp:
    for line in fp:

        if line.strip() == "embeddings = inputs_embeds + position_embeddings  # (bs, max_seq_length, dim)":

            line_modified = "        embeddings = inputs_embeds + tf.cast(position_embeddings, dtype=inputs_embeds.dtype)  # (bs, max_seq_length, dim)" + "\n"
            lines.append(line_modified)

        else:

            lines.append(line)

with open(python_lib_path + '/site-packages/transformers/modeling_tf_distilbert.py', "w", encoding="UTF-8") as fp:
    for line in lines:
        fp.write(line)


lines = []
with open(python_lib_path + '/site-packages/transformers/modeling_tf_distilbert.py', "r", encoding="UTF-8") as fp:
    for line in fp:

        if line.strip() == "cdf = 0.5 * (1.0 + tf.math.erf(x / tf.math.sqrt(2.0)))":

            line_modified = "    cdf = tf.cast(0.5, dtype=x.dtype) * (tf.cast(1.0, dtype=x.dtype) + tf.math.erf(x / tf.cast(tf.math.sqrt(2.0), dtype=x.dtype)))" + "\n"
            lines.append(line_modified)

        else:

            lines.append(line)

with open(python_lib_path + '/site-packages/transformers/modeling_tf_distilbert.py', "w", encoding="UTF-8") as fp:
    for line in lines:
        fp.write(line)

In [6]:
import os
import time
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
from transformers import *
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
import random
from kaggle_datasets import KaggleDatasets
import transformers
import tensorflow as tf

from sklearn.metrics import accuracy_score, roc_auc_score

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn import metrics
from sklearn.utils import shuffle

pd.set_option('max_colwidth',500)
np.random.seed(0)

In [8]:
from tensorflow.keras.mixed_precision import experimental as mixed_precision
policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
mixed_precision.set_policy(policy)

In [10]:
AUTO = tf.data.experimental.AUTOTUNE
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)
print(strategy.num_replicas_in_sync)
BATCH_SIZE = 44 * strategy.num_replicas_in_sync
GCS_DS_PATH = KaggleDatasets().get_gcs_path('jigsaw-multilingual-toxic-comment-classification')
maxlen = 192
MODEL = '/kaggle/input/jplu-tf-xlm-roberta-large'

8


In [11]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/encoded-435/__results__.html
/kaggle/input/encoded-435/x_train.npy
/kaggle/input/encoded-435/y_train.npy
/kaggle/input/encoded-435/x_test_google.npy
/kaggle/input/encoded-435/x_test_en_blob.npy
/kaggle/input/encoded-435/__notebook__.ipynb
/kaggle/input/encoded-435/custom.css
/kaggle/input/encoded-435/y_valid.npy
/kaggle/input/encoded-435/__output__.json
/kaggle/input/encoded-435/x_test_yandex.npy
/kaggle/input/encoded-435/x_test.npy
/kaggle/input/encoded-435/x_valid.npy
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-cl

In [12]:
test = pd.read_csv("/kaggle/input/jigsaw-data/test_all.csv")

In [13]:
# loaded from encoded-435 file
from numpy import load
x_train = load('/kaggle/input/encoded-435/x_train.npy').astype(np.float32)
x_valid = load('/kaggle/input/encoded-435/x_valid.npy').astype(np.float32)
x_test = load('/kaggle/input/encoded-435/x_test.npy').astype(np.float32)

x_test_google = load('/kaggle/input/encoded-435/x_test_google.npy').astype(np.float32)
x_test_yandex =load('/kaggle/input/encoded-435/x_test_yandex.npy').astype(np.float32)
x_test_en_blob = load('/kaggle/input/encoded-435/x_test_en_blob.npy').astype(np.float32)

y_valid = load('/kaggle/input/encoded-435/y_valid.npy').astype(np.float32)

y_train = load('/kaggle/input/encoded-435/y_train.npy').astype(np.float32)

In [14]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

test_dataset_google = (
    tf.data.Dataset
    .from_tensor_slices(x_test_google)
    .batch(BATCH_SIZE)
)

test_dataset_yandex = (
    tf.data.Dataset
    .from_tensor_slices(x_test_yandex)
    .batch(BATCH_SIZE)
)

test_dataset_en_blob = (
    tf.data.Dataset
    .from_tensor_slices(x_test_en_blob)
    .batch(BATCH_SIZE)
)

In [15]:
def build_model(transformer, loss='binary_crossentropy', max_len=192):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    out = Dense(1)(x)
    out = layers.Activation('sigmoid', dtype='float32', name='predictions')(out)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1.6e-5), loss=loss, metrics=["accuracy",tf.keras.metrics.AUC()])
    
    return model

In [18]:
%%time
#prepare model for training on TPU
with strategy.scope():
    transformer_layer = transformers.TFXLMRobertaModel.from_pretrained(MODEL)
    model = build_model(transformer_layer,loss='binary_crossentropy', max_len=maxlen)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tfxlm_roberta_model (TFXLMRo ((None, 192, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dropout_74 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
_________________________________________________________________
predictions (Activation)     (None, 1)                 0         
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_______________________________________________

In [19]:
#train model
N_STEPS = x_train.shape[0] // BATCH_SIZE
EPOCHS = 2
train_history = model.fit(
    train_dataset,
    steps_per_epoch=N_STEPS,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Train for 1237 steps, validate for 23 steps
Epoch 1/2
1237/1237 [==============================] - 1435s 1s/step - loss: 0.2042 - accuracy: 0.9140 - auc: 0.9699 - val_loss: 0.3208 - val_accuracy: 0.8808 - val_auc: 0.9003
Epoch 2/2
1237/1237 [==============================] - 1130s 914ms/step - loss: 0.1480 - accuracy: 0.9396 - auc: 0.9838 - val_loss: 0.3460 - val_accuracy: 0.8753 - val_auc: 0.8927


In [20]:
model.save_weights('model_val_untrained.h5')

In [21]:
# train on valid set
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs= EPOCHS
)

Train for 22 steps
Epoch 1/2
22/22 [==============================] - 185s 8s/step - loss: 0.2626 - accuracy: 0.8802 - auc: 0.9068
Epoch 2/2
22/22 [==============================] - 122s 6s/step - loss: 0.1810 - accuracy: 0.9222 - auc: 0.9576


In [22]:
model.save_weights('model.h5')

In [23]:
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')
sub_fin = sub.copy()
sub['toxic_o'] = model.predict(test_dataset, verbose=1)

182/182 [==============================] - 120s 661ms/step


In [24]:
sub['toxic_google'] = model.predict(test_dataset_google, verbose=1)

182/182 [==============================] - 70s 384ms/step


In [25]:
sub['toxic_yandex'] = model.predict(test_dataset_yandex, verbose=1)

182/182 [==============================] - 61s 334ms/step


In [26]:
sub['toxic_en_blob'] = model.predict(test_dataset_en_blob, verbose=1)

182/182 [==============================] - 62s 342ms/step


In [27]:
pass1 = test[test.en_blob.str.find("pass") == 0]

In [28]:
for i in pass1.id:
    if i in sub.id:
        sub.toxic_en_blob.loc[i] = (sub.toxic_google.loc[i] + sub.toxic_yandex.loc[i] + sub.toxic_o.loc[i]) / 3

In [29]:
sub_fin['toxic'] = (0.6*sub['toxic_o']) + (0.1*sub['toxic_google']) + (0.15*sub['toxic_yandex']) + (0.15*sub['toxic_en_blob'])

In [30]:
sub.to_csv('submission_all.csv', index=False)
sub_fin.to_csv('submission.csv',index=False)